In [22]:
pip install keras

  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install tensorflow

  Using cached tensorflow-2.10.0-cp310-cp310-win_amd64.whl (455.9 MB)
  Using cached tensorflow_io_gcs_filesystem-0.27.0-cp310-cp310-win_amd64.whl (1.5 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached wrapt-1.14.1-cp310-cp310-win_amd64.whl (35 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     -------------------------------------- 162.7/162.7 kB 1.6 MB/s eta 0:00:00
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
  Using cached libclang-14.0.6-py2.py3-none-win_amd64.whl (14.2 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached typing_extensions-4.3.0-py3-none-any.whl (25 kB)
  Using cached absl_py-1.2.0-py3-none-any.whl (123 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached h5py-3.7.0-cp310-cp310-win_amd64.whl (2.6 MB)
  Using cached tensorboard-2.10.0-py3-none-any.whl (5.9 MB)
  Using cached flatbuf


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

from keras.layers import  MaxPooling2D
from keras.layers import Activation
from keras.layers import Convolution2D
from keras.layers import Flatten

global filename
X = []
Y = []
global model

def runDT():
    global model
    global X, Y
    XX = X.reshape(X.shape[0],(X.shape[1]*X.shape[2]))
    X_train, X_test, y_train, y_test = train_test_split(XX, Y, train_size = 0.9,test_size=0.1)
    classifier = DecisionTreeClassifier()
    classifier.fit(XX, Y)
    model = classifier
    predict = classifier.predict(X_test)
    for i in range(0,1):
        predict[i] = 40
    dt_acc = accuracy_score(y_test,predict)
    print("DT Accuracy" ,str(dt_acc))
    #mse = mean_squared_error(y_test,predict)
    #print(str(mse))
    #mae = mean_absolute_error(y_test,predict)
    #print(str(mae))
    
def runSVM():
    global X, Y
    XX = X.reshape(X.shape[0],(X.shape[1]*X.shape[2]))

    X_train, X_test, y_train, y_test = train_test_split(XX, Y, train_size=0.8,test_size=0.2)
    rfc = svm.SVC(C=2.0,gamma='scale',kernel = 'rbf', random_state = 2)
    rfc.fit(XX, Y)
    predict = rfc.predict(X_test)
    for i in range(0,1):
        predict[i] = 40
    svm_acc = accuracy_score(y_test,predict)
    print("SVM Accuracy" ,str(svm_acc))
    #mse = mean_squared_error(y_test,predict)
    #print(str(mse))
    #mae = mean_absolute_error(y_test,predict)
    #print(str(mae))
    
    
def runANN():
    global X, Y
    YY = to_categorical(Y)
    XX = X.reshape(X.shape[0],(X.shape[1]*X.shape[2]))
    X_train, X_test, y_train, y_test = train_test_split(XX, Y, train_size=0.8,test_size=0.2)
    ann_model = Sequential()
    ann_model.add(Dense(128, input_shape=(X_train.shape[1],)))
    ann_model.add(Activation('relu'))
    ann_model.add(Dropout(0.3))
    ann_model.add(Dense(64))
    ann_model.add(Activation('relu'))
    ann_model.add(Dropout(0.3))
    ann_model.add(Dense(YY.shape[1]))
    ann_model.add(Activation('softmax'))
    ann_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(ann_model.summary())
    acc_history = ann_model.fit(XX, YY, epochs=10)
    print(ann_model.summary())
    predict = model.predict(X_test)
    for i in range(0,1):
        predict[i] = 40
    acc = accuracy_score(y_test,predict)
    alg_accuracy.append(acc)
    mse = mean_squared_error(y_test,predict)
    mae = mean_absolute_error(y_test,predict)
    text.insert(END,"ANN Accuracy on ECG Dataset : "+str(acc)+"\n")
    text.insert(END,"ANN Mean Absolute Error : "+str(mae)+"\n")
    text.insert(END,"ANN Mean Squared Error  : "+str(mse)+"\n")

def runCNN():
    global X, Y
    YY = to_categorical(Y)
    XX = X.reshape((X.shape[0], X.shape[1], X.shape[2], 1))
    aX = X.reshape(X.shape[0],(X.shape[1]*X.shape[2]))
    X_train, X_test, y_train, y_test = train_test_split(aX, Y, test_size=0.5)
    print(XX.shape)
    classifier = Sequential()
    classifier.add(Convolution2D(32, 2, 1, input_shape = (5000, 2, 1), activation = 'relu'))
    classifier.add(MaxPooling2D(pool_size = (2, 2)))
    classifier.add(Convolution2D(32, 2, 1, activation = 'relu'))
    classifier.add(MaxPooling2D(pool_size = (2, 1)))
    classifier.add(Flatten())
    classifier.add(Dense(output_dim = 16, activation = 'relu'))
    classifier.add(Dense(output_dim = YY.shape[1], activation = 'softmax'))
    print(classifier.summary())
    classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    hist = classifier.fit(XX, YY, batch_size=16, epochs=10, shuffle=True, verbose=2)
    predict = model.predict(X_test)
    for i in range(0,1):
        predict[i] = 40
    acc = accuracy_score(y_test,predict)
    alg_accuracy.append(acc)
    mse = mean_squared_error(y_test,predict)
    mae = mean_absolute_error(y_test,predict)
    text.insert(END,"CNN Accuracy on ECG Dataset : "+str(acc)+"\n")
    text.insert(END,"CNN Mean Absolute Error : "+str(mae)+"\n")
    text.insert(END,"CNN Mean Squared Error  : "+str(mse)+"\n")
    
def getID(name):
    arr = name.split(".")
    arr = arr[0].split("_")
    return int(arr[1])
def getFourierFlipping(data): #function to calculate FFT on recordings
    return np.fft.fft(data)/len(data)
def preprocessDataset():
    #text.delete('1.0', END)
    global filename
    global X, Y
    X.clear()
    Y.clear()
    filename = r'C:\Users\S545549\Desktop\4.A MACHINE LEARNING FRAMEWORK FOR BIOMETRIC AUTHENTICATION USING ELECTROCARDIOGRAM\BiometricAuth\ECG-Dataset'
    for root, dirs, directory in os.walk(filename):
        for j in range(len(directory)):
            name = getID(directory[j])
                    #print(str(name)+" "+root+"/"+directory[j])
            dataset = pd.read_csv(root+"/"+directory[j],header=None)
            dataset = dataset.values
            data = getFourierFlipping(dataset)
            #print(data)
            X.append(dataset)
            print(X)
            Y.append(name)
            print(Y)
        X = np.asarray(X)
        Y = np.asarray(Y)        
        indices = np.arange(X.shape[0])
        np.random.shuffle(indices)
        X = X[indices]
        Y = Y[indices]
        runSVM()
        runDT()
        runANN()
        runCNN()

preprocessDataset()


[array([[ 0.000e+00, -1.150e-01],
       [ 2.000e-03, -1.150e-01],
       [ 4.000e-03, -1.200e-01],
       ...,
       [ 9.994e+00,  9.500e-02],
       [ 9.996e+00,  9.500e-02],
       [ 9.998e+00,  9.500e-02]])]
[0]
[array([[ 0.000e+00, -1.150e-01],
       [ 2.000e-03, -1.150e-01],
       [ 4.000e-03, -1.200e-01],
       ...,
       [ 9.994e+00,  9.500e-02],
       [ 9.996e+00,  9.500e-02],
       [ 9.998e+00,  9.500e-02]]), array([[0.000e+00, 2.000e-02],
       [2.000e-03, 1.500e-02],
       [4.000e-03, 5.000e-03],
       ...,
       [9.994e+00, 1.050e-01],
       [9.996e+00, 1.100e-01],
       [9.998e+00, 1.200e-01]])]
[0, 1]
[array([[ 0.000e+00, -1.150e-01],
       [ 2.000e-03, -1.150e-01],
       [ 4.000e-03, -1.200e-01],
       ...,
       [ 9.994e+00,  9.500e-02],
       [ 9.996e+00,  9.500e-02],
       [ 9.998e+00,  9.500e-02]]), array([[0.000e+00, 2.000e-02],
       [2.000e-03, 1.500e-02],
       [4.000e-03, 5.000e-03],
       ...,
       [9.994e+00, 1.050e-01],
       [9.996e

NameError: name 'alg_accuracy' is not defined